In [7]:
import numpy as np
import findspark
from functions.required_functions import readFile, normalize, transform, get_block_data, train, accuracy
from pyspark.sql import SparkSession
findspark.init()

In [8]:
spark = SparkSession.builder.master("local[*]").appName('Exercise_1').getOrCreate()
sc = spark.sparkContext

26/02/21 12:35:58 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [ ]:
rdd = readFile("data/botnet_reduced_10k_l.csv", sc)
TOTAL = rdd.count()
#rdd.take(2)

[((3545.3018916840147,
   3198.0139469522546,
   80.00015469454229,
   1.0000019205086303,
   444960086.4643011,
   476.8073064086366,
   12.999999992936175,
   -7.710424743123667e-09,
   87.00000148917339,
   19099430.15956688,
   2468368394.7593513),
  0),
 ((-1.21308761436012e-06,
   7.999989079213265,
   65499.99887793755,
   0.9999997452701503,
   61.99988768910407,
   69.99980788311223,
   11.999999995904211,
   -2.35688187855132e-08,
   8.00000498340475,
   2468369617.006896,
   2468368392.7377276),
  1)]

In [11]:
normalize(rdd).take(2)

[(array([ 1.41281668, -0.75956071, -0.41204268, -0.45940748,  1.39580934,
         -0.35386218,  0.74057161, -0.8945781 , -0.40163083, -2.91039199,
          0.1567098 ]),
  0),
 (array([-0.79867306, -0.89146231,  3.64034447, -0.45940748, -0.52296869,
         -0.35387136,  0.55184122, -0.8945781 , -1.27444226,  0.4762026 ,
          0.1567098 ]),
  1)]

In [ ]:
path = "data/botnet_reduced_10k_l.csv"  # botnet_tot_syn_l.csv // botnet_reduced_10k_l.csv
nIter = 20
learning_rate =  1.5
lambda_reg = 0

# Read data
data = readFile(path,sc).cache()
print(f"Total samples: {data.count()}")

# Standardize
data = normalize(data)
print("Data normalized.")

sums, n = data.map(lambda x: (x[0], 1)).reduce(lambda x, y: (x[0] + y[0], x[1] + y[1]))
medias = sums / n
print(f"Feature means after normalization: {np.array2string(medias, precision=4, suppress_small=True)}")
desvs = (data.map(lambda x: (x[0]-medias)**2).reduce(lambda x, y: x + y) / (n-1))**0.5
print(f"Feature stds after normalization: {np.array2string(desvs, precision=4, suppress_small=True)}")

w_b = train(data, nIter, learning_rate, lambda_reg)
acc = accuracy(w_b[0], w_b[1], data)
print(f"Accuracy: {acc:.4f}")

Total samples: 10000
Data normalized.
Feature means after normalization: [ 0.  0. -0.  0. -0. -0.  0. -0. -0.  0. -0.]
Feature stds after normalization: [1.0001 1.0001 1.0001 1.0001 1.0001 1.0001 1.0001 1.0001 1.0001 1.0001
 1.0001]
Iteration 0/20, Cost: 0.5323, Accuracy: 0.7408
Iteration 2/20, Cost: 0.3166, Accuracy: 0.9043
Iteration 4/20, Cost: 0.2617, Accuracy: 0.9193
Iteration 6/20, Cost: 0.2364, Accuracy: 0.9258
Iteration 8/20, Cost: 0.2216, Accuracy: 0.9280
Iteration 10/20, Cost: 0.2118, Accuracy: 0.9292
Iteration 12/20, Cost: 0.2047, Accuracy: 0.9310
Iteration 14/20, Cost: 0.1994, Accuracy: 0.9325
Iteration 16/20, Cost: 0.1953, Accuracy: 0.9332
Iteration 18/20, Cost: 0.1920, Accuracy: 0.9336
Accuracy: 0.9336


In [14]:
sc.stop()